In [482]:
import glob
# This file should run basically insta fast. Bad coding for sure 
genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = "02" # Which month and days result you want to visualize in table
month = 11
repetitionValue = 46
repetitionName = "udrlBC3special005"
repetitionString = f"{repetitionValue}{repetitionName}"
curSavePath = f"{genSavePath}{day}-{month}-{repetitionString}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

skipSubjects = []
onlySignificant = True
minAcc = 0.5
print(len(savedResults))
print(len(savedResults)/9)

5
0.5555555555555556


In [483]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                if ares > 0.98:
                    ares = 0.99
                akernel = aResults2[2]
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                  
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
topThreeAllSubjects = []
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    topThreeThisSubject = []
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()

            
            
            if len(topThreeThisSubject) < 2 or topThreeThisSubject[0][0] < res[1]:

                if len(topThreeThisSubject) == 2:
                    topThreeThisSubject.reverse()
                    topThreeThisSubject.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topThreeThisSubject.append([res[1], subNr, dataF, kernel, cVal])
                topThreeThisSubject.sort()
    if len(topThreeThisSubject)>0:
        topThreeAllSubjects.append(topThreeThisSubject)
# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
subjectResultArray = np.array(subjectResultArray)
emptyCols = []
atleastOneEmpty = False
for hcol, in zip(headers):
    if all(subjectResultArray[:,hcol] == None):
        atleastOneEmpty = True
        emptyCols.append(hcol)
if atleastOneEmpty: 
    subjectResultArray = np.delete(subjectResultArray, np.array(emptyCols),1)
    headers = np.delete(headers, np.array(emptyCols),0)
    comboAverage = np.delete(comboAverage, np.array(emptyCols),1)
    comboTotal = np.delete(comboTotal, np.array(emptyCols),1)
    nrOfSubjects = np.delete(nrOfSubjects, np.array(emptyCols),0)


table = pd.DataFrame(subjectResultArray, columns=headers)
display(table)




(1, 189)
(10, 189)


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_16776\1128782281.py:188: RuntimeWarning: invalid value encountered in divide
  comboAverage = comboAverage / nrOfSubjects


,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,24,25,26,27,28,29,30,31,32,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,0.625,0.725,0.6,0.775,0.75,0.775,0.575,0.725,0.675,0.5,0.725,0.575,0.65,0.6,0.625,0.625,0.725,0.6,0.775,0.75,0.775,0.6,0.75,0.675,0.65,0.6,0.625,0.625,0.725,0.6,0.775,0.75,0.775,0.575,0.725,0.675,0.65,None,None,0.65,0.6,0.625,0.75,0.75,0.775,0.675,0.725,0.65,0.625,0.7,0.675,0.675,0.725,0.7,0.7,0.75,0.775,0.775,0.725,0.675,0.7,0.65,0.7,0.65,0.675,0.65,0.675,0.725,0.7,0.625,0.675,0.725,0.625,0.725,0.6,0.775,0.75,0.775,0.6,0.75,0.675,0.65,0.6,0.625,0.75,0.75,0.775,0.675,0.725,0.65,0.625,0.7,0.675,0.675,0.725,0.7,0.675,0.75,0.8,0.775,0.725,0.675,0.7,0.65,0.725,0.6,0.675,0.625,0.675,0.725,0.7,0.65,0.625,0.7,0.75,0.75,0.775,0.7,0.725,0.65,0.625,0.7,0.675,0.675,0.725,0.7,0.7,0.75,0.75,0.775,0.725,0.675,0.7,0.65,0.725,0.575,0.675,0.625,0.675,0.725,0.7,0.65,0.625,0.7,0.75,0.775,0.725,0.75,0.75,0.7,0.7,0.775,0.75,0.675,0.675,0.675,0.7,0.725,0.7,0.675,0.725,0.7,0.7,0.75,0.7,0.725,0.7,0.775,0.7,0.675,0.675,0.65,0.75,0.7
2,0.791667,0.875,0.791667,0.75,0.75,0.791667,0.729167,0.6875,0.6875,0.666667,0.770833,0.6875,0.770833,0.770833,0.770833,0.791667,0.875,0.791667,0.75,0.75,0.791667,0.708333,0.6875,0.6875,0.770833,0.770833,0.770833,0.791667,0.875,0.791667,0.75,0.729167,0.791667,0.75,0.6875,0.6875,0.770833,None,None,0.770833,0.770833,0.770833,0.770833,0.833333,0.75,0.770833,0.854167,0.6875,0.791667,0.833333,0.6875,0.75,0.8125,0.75,0.75,0.770833,0.8125,0.75,0.770833,0.708333,0.729167,0.770833,0.791667,0.75,0.791667,0.6875,0.791667,0.770833,0.791667,0.729167,0.791667,0.75,0.791667,0.875,0.791667,0.75,0.729167,0.791667,0.729167,0.6875,0.6875,0.770833,0.770833,0.770833,0.770833,0.833333,0.75,0.770833,0.854167,0.6875,0.791667,0.833333,0.6875,0.75,0.8125,0.75,0.75,0.770833,0.8125,0.75,0.770833,0.708333,0.729167,0.770833,0.791667,0.708333,0.6875,0.666667,0.791667,0.770833,0.791667,0.770833,0.791667,0.708333,0.770833,0.833333,0.75,0.75,0.854167,0.6875,0.791667,0.833333,0.6875,0.75,0.8125,0.75,0.75,0.770833,0.8125,0.75,0.770833,0.708333,0.729167,0.770833,0.791667,0.75,0.6875,0.666667,0.791667,0.770833,0.791667,0.770833,0.791667,0.708333,0.833333,0.8125,0.8125,0.770833,0.791667,0.75,0.791667,0.833333,0.791667,0.770833,0.854167,0.666667,0.75,0.833333,0.770833,0.75,0.8125,0.6875,0.75,0.770833,0.6875,0.75,0.8125,0.770833,0.729167,0.791667,0.708333,0.770833,0.791667,0.770833
3,0.555556,0.694444,0.527778,0.777778,0.75,0.722222,0.722222,0.722222,0.666667,0.694444,0.694444,0.75,0.666667,0.694444,0.694444,0.555556,0.694444,0.527778,0.777778,0.75,0.722222,0.722222,0.722222,0.666667,0.666667,0.694444,0.694444

In [484]:
print("Top ten highest accuracies")

topTen.reverse()
for top in topTen:
    print(
            f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
        )


Top ten highest accuracies
Accuracy 0.88 : subject 2, feature:dataFFTCV-BC-dataFFTCV--fftData, kernelrbf, C:2.5
Accuracy 0.88 : subject 2, feature:dataFFTCV-BC--fftData, kernelrbf, C:2.5
Accuracy 0.88 : subject 2, feature:dataFFTCV-BC--dataFFTCV, kernelrbf, C:2.5
Accuracy 0.88 : subject 2, feature:-dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.85 : subject 2, feature:dataWCV-BCcn6-dataCVBC--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.85 : subject 2, feature:dataCVBC-dataFFTCV-BC--fftData, kernelrbf, C:2.5
Accuracy 0.85 : subject 2, feature:dataCVBC-dataFFTCV-BC--dataFFTCV, kernelrbf, C:2.5
Accuracy 0.85 : subject 2, feature:dataCVBC--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.83 : subject 2, feature:fftDataBC-dataFFTCV-BC--fftData, kernelrbf, C:2.5
Accuracy 0.83 : subject 2, feature:fftDataBC--dataFFTCV-BC, kernelrbf, C:2.5


In [485]:
print("Top ten highest accuracies")
#del(topThreeThisSubject[0])
#len(topThreeAllSubjects[0])
for subTop in topThreeAllSubjects:
    subTop.reverse()
topThreeAllSubjects.sort(reverse=True)
for subTop in topThreeAllSubjects:
    print("")
    #subTop.reverse()
    for top in subTop:
        print(
                f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
            )

Top ten highest accuracies

Accuracy 0.88 : subject 2, feature:dataFFTCV-BC--fftData, kernelrbf, C:2.5
Accuracy 0.88 : subject 2, feature:-dataFFTCV-BC, kernelrbf, C:2.5

Accuracy 0.83 : subject 5, feature:welchDatacn6BC-fftDataBC--fftData, kernellinear, C:0.5
Accuracy 0.83 : subject 5, feature:welchDatacn6BC--fftDataBC, kernellinear, C:0.5

Accuracy 0.83 : subject 4, feature:fftDataBC-dataFFTCV-BC--fftData, kernelsigmoid, C:2.5
Accuracy 0.81 : subject 4, feature:fftDataBC--fftData, kernelrbf, C:2.5

Accuracy 0.83 : subject 3, feature:fftDataBC-dataFFTCV-BC--fftData, kernelsigmoid, C:2.5
Accuracy 0.83 : subject 3, feature:fftDataBC--dataFFTCV-BC, kernelsigmoid, C:2.5

Accuracy 0.8 : subject 1, feature:dataCVBC-fftDataBC--fftData, kernelsigmoid, C:2.5
Accuracy 0.78 : subject 1, feature:-fftDataBC, kernelsigmoid, C:2.5


In [486]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)

print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
print(headers[maxInd])
print(nrOfSubjects[maxInd])
print(list(nameDict.keys())[combinationList[maxInd][0]])
print(list(kernelDict.keys())[combinationList[maxInd][1]])
print(f"C: {combinationList[maxInd][2]}")


[0.         0.69229651 0.76392926 0.68992248 0.72772158 0.72689176
 0.         0.         0.         0.        ]


,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,24,25,26,27,28,29,30,31,32,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188
0,0.661111,0.742222,0.646389,0.768889,0.754167,0.749444,0.655278,0.710278,0.685,0.618056,0.692222,0.635833,0.688333,0.700556,0.713889,0.652778,0.742222,0.646389,0.768889,0.754167,0.749444,0.639444,0.715278,0.685,0.688333,0.700556,0.713889,0.679167,0.746389,0.642222,0.768889,0.75,0.745278,0.674722,0.714444,0.685,0.706389,0.78125,0.75,0.6925,0.700556,0.713889,0.747222,0.736111,0.784167,0.700278,0.738056,0.699444,0.669444,0.7275,0.680833,0.718333,0.711667,0.723333,0.737222,0.752778,0.774444,0.768889,0.751944,0.735,0.748333,0.714722,0.737222,0.695278,0.712778,0.678611,0.700278,0.728333,0.751111,0.679167,0.714167,0.711667,0.670833,0.746389,0.642222,0.768889,0.75,0.745278,0.6575,0.719444,0.680833,0.6925,0.700556,0.713889,0.747222,0.736111,0.788333,0.700278,0.738056,0.699444,0.652778,0.719167,0.682222,0.718333,0.711667,0.723333,0.737778,0.752778,0.779444,0.768889,0.751944,0.730833,0.748333,0.714722,0.746389,0.639444,0.691944,0.672222,0.704444,0.728333,0.751111,0.688333,0.722222,0.716389,0.747222,0.736111,0.784167,0.7025,0.733889,0.699444,0.679167,0.7275,0.679444,0.718333,0.711667,0.7275,0.737222,0.752778,0.769444,0.768889,0.747778,0.735,0.748333,0.714722,0.742222,0.684444,0.696111,0.670833,0.704444,0.7325,0.751111,0.686944,0.722222,0.726111,0.768056,0.7425,0.761667,0.747222,0.7375,0.748333,0.74,0.756389,0.756944,0.696111,0.723889,0.711389,0.721944,0.733889,0.744167,0.718333,0.72,0.723333,0.737222,0.765278,0.730278,0.750556,0.753889,0.761944,0.748333,0.741944,0.729444,0.686944,0.738889,0.728889


Max average accuracy 0.7883333333333333
101
5.0
dataFFTCV-BC-dataFFTCV--fftData
sigmoid
C: 2.5


In [487]:
x = 0
for names in nameDict.keys():
    print(names)
    x +=1
    print(x)

-fftData
1
-dataFFTCV
2
-dataFFTCV-BC
3
-fftDataBC
4
-dataCVBC
5
-dataWCV-BCcn6
6
-welchDatacn6BC
7
dataFFTCV--fftData
8
dataFFTCV-BC--fftData
9
fftDataBC--fftData
10
dataCVBC--fftData
11
dataWCV-BCcn6--fftData
12
welchDatacn6BC--fftData
13
dataFFTCV-BC--dataFFTCV
14
fftDataBC--dataFFTCV
15
dataCVBC--dataFFTCV
16
dataWCV-BCcn6--dataFFTCV
17
welchDatacn6BC--dataFFTCV
18
fftDataBC--dataFFTCV-BC
19
dataCVBC--dataFFTCV-BC
20
dataWCV-BCcn6--dataFFTCV-BC
21
welchDatacn6BC--dataFFTCV-BC
22
dataCVBC--fftDataBC
23
dataWCV-BCcn6--fftDataBC
24
welchDatacn6BC--fftDataBC
25
dataWCV-BCcn6--dataCVBC
26
welchDatacn6BC--dataCVBC
27
welchDatacn6BC--dataWCV-BCcn6
28
dataFFTCV-BC-dataFFTCV--fftData
29
fftDataBC-dataFFTCV--fftData
30
dataCVBC-dataFFTCV--fftData
31
dataWCV-BCcn6-dataFFTCV--fftData
32
welchDatacn6BC-dataFFTCV--fftData
33
fftDataBC-dataFFTCV-BC--fftData
34
dataCVBC-dataFFTCV-BC--fftData
35
dataWCV-BCcn6-dataFFTCV-BC--fftData
36
welchDatacn6BC-dataFFTCV-BC--fftData
37
dataCVBC-fftDataBC--fftDa

In [488]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(headers[maxInd2])
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,24,25,26,27,28,29,30,31,32,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188
0,3.305556,3.711111,3.231944,3.844444,3.770833,3.747222,3.276389,3.551389,3.425,3.090278,3.461111,3.179167,3.441667,3.502778,3.569444,3.263889,3.711111,3.231944,3.844444,3.770833,3.747222,3.197222,3.576389,3.425,3.441667,3.502778,3.569444,3.395833,3.731944,3.211111,3.844444,3.75,3.726389,3.373611,3.572222,3.425,3.531944,1.5625,0.75,3.4625,3.502778,3.569444,3.736111,3.680556,3.920833,3.501389,3.690278,3.497222,3.347222,3.6375,3.404167,3.591667,3.558333,3.616667,3.686111,3.763889,3.872222,3.844444,3.759722,3.675,3.741667,3.573611,3.686111,3.476389,3.563889,3.393056,3.501389,3.641667,3.755556,3.395833,3.570833,3.558333,3.354167,3.731944,3.211111,3.844444,3.75,3.726389,3.2875,3.597222,3.404167,3.4625,3.502778,3.569444,3.736111,3.680556,3.941667,3.501389,3.690278,3.497222,3.263889,3.595833,3.411111,3.591667,3.558333,3.616667,3.688889,3.763889,3.897222,3.844444,3.759722,3.654167,3.741667,3.573611,3.731944,3.197222,3.459722,3.361111,3.522222,3.641667,3.755556,3.441667,3.611111,3.581944,3.736111,3.680556,3.920833,3.5125,3.669444,3.497222,3.395833,3.6375,3.397222,3.591667,3.558333,3.6375,3.686111,3.763889,3.847222,3.844444,3.738889,3.675,3.741667,3.573611,3.711111,3.422222,3.480556,3.354167,3.522222,3.6625,3.755556,3.434722,3.611111,3.630556,3.840278,3.7125,3.808333,3.736111,3.6875,3.741667,3.7,3.781944,3.784722,3.480556,3.619444,3.556944,3.609722,3.669444,3.720833,3.591667,3.6,3.616667,3.686111,3.826389,3.651389,3.752778,3.769444,3.809722,3.741667,3.709722,3.647222,3.434722,3.694444,3.644444


101
5.0
Max total significant accuracy then averaged 0.7883333333333333 
dataFFTCV-BC-dataFFTCV--fftData
sigmoid
C: 2.5 


In [489]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
